# Data Wrangling Exercises

### 1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.

- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.

- Only include properties that include a latitude and longitude value.

In [1]:
import pandas as pd
import numpy as np
import os
from env import host, user, password

def get_connection(db, user=user, host=host, password=password):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    It takes in a string name of a database as an argument.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
    
def new_zillow_data():
    '''
    This function reads all tables and columns into a dataframe, including only properties with lat/long
    data and had transactions in 2017. Properties with multiple transactions 
    will display only the most recent transaction.
    '''
    sql_query = """
                SELECT *
                FROM properties_2017 as p
                JOIN predictions_2017 as m USING(parcelid)
                LEFT JOIN airconditioningtype as ac USING(airconditioningtypeid)
                LEFT JOIN architecturalstyletype as arch USING(architecturalstyletypeid)
                LEFT JOIN buildingclasstype as b USING(buildingclasstypeid)
                LEFT JOIN heatingorsystemtype as heat USING(heatingorsystemtypeid)
                LEFT JOIN propertylandusetype as pt USING(propertylandusetypeid)
                LEFT JOIN storytype as s USING(storytypeid)
                LEFT JOIN typeconstructiontype as c USING(typeconstructiontypeid)
                LEFT JOIN unique_properties as u USING(parcelid)
                INNER JOIN (SELECT p.id, MAX(transactiondate) AS maxdate FROM properties_2017 as p JOIN predictions_2017 USING(parcelid) GROUP BY p.id) md ON p.id = md.id AND transactiondate = maxdate
                WHERE transactiondate LIKE '2017%%' AND latitude IS NOT NULL AND longitude IS NOT NULL
                """
    
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('zillow'))
    
    return df

def get_zillow_data():
    '''
    This function reads in zillow data from the Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('zillow.csv'):
        
        # If csv file exists read in data from csv file.
        df = pd.read_csv('zillow.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame
        df = new_zillow_data()
        
        # Cache data
        df.to_csv('zillow.csv')
        
    return df


### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [2]:
df = get_zillow_data()

/opt/homebrew/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (65) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
cols = []
count = 1
for column in df.columns:
    if column == 'id':
        cols.append(f'id_{count}')
        count+=1
        continue
    cols.append(column)
df.columns = cols

In [4]:
df.drop(columns = ['id.2','id.1','maxdate'],inplace = True)

In [5]:
df.rename(columns = {'id_1':'id'}, inplace = True)

In [6]:
for col in df.columns:
    print(df[col].value_counts())
    print('\n---------------------------')

11933699    1
11505743    1
11709951    1
11445758    1
14728701    1
           ..
12792914    1
13840735    1
12083865    1
14183064    1
12853244    1
Name: parcelid, Length: 77380, dtype: int64

---------------------------
6.0     219
4.0       1
13.0      1
10.0      1
Name: typeconstructiontypeid, dtype: int64

---------------------------
7.0    50
Name: storytypeid, dtype: int64

---------------------------
261.0    52319
266.0    19294
246.0     2009
269.0     1944
248.0      727
247.0      535
265.0      333
263.0       74
275.0       58
260.0       37
267.0       29
31.0        15
264.0        6
Name: propertylandusetypeid, dtype: int64

---------------------------
2.0     33550
7.0     14064
24.0      846
6.0       776
20.0      104
13.0       59
18.0       23
1.0        13
10.0        3
11.0        1
Name: heatingorsystemtypeid, dtype: int64

---------------------------
4.0    14
3.0     1
Name: buildingclasstypeid, dtype: int64

---------------------------
7.0     172
8.0 

In [7]:
df.shape

(77380, 68)

In [8]:
df.dtypes

parcelid                    int64
typeconstructiontypeid    float64
storytypeid               float64
propertylandusetypeid     float64
heatingorsystemtypeid     float64
                           ...   
buildingclassdesc          object
heatingorsystemdesc        object
propertylandusedesc        object
storydesc                  object
typeconstructiondesc       object
Length: 68, dtype: object

In [9]:
df.describe()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,7.738000e+04,222.000000,50.0,77380.000000,49439.000000,15.000000,206.000000,24953.000000,7.738000e+04,50.000000,...,17560.000000,172.0,7.726800e+04,7.737900e+04,77380.0,7.737800e+04,77375.000000,2886.000000,7.713600e+04,77380.000000
mean,1.300715e+07,6.040541,7.0,261.826350,3.920447,3.933333,7.388350,1.813289,1.495126e+06,679.720000,...,1.434738,1.0,1.893160e+05,4.901375e+05,2016.0,3.010969e+05,5995.566592,14.087318,6.049674e+13,0.016628
std,3.481368e+06,0.557285,0.0,5.141231,3.592789,0.258199,2.734542,2.967894,8.609057e+05,689.703546,...,0.544569,0.0,2.300886e+05,6.534477e+05,0.0,4.925990e+05,7622.887460,2.185663,1.535252e+12,0.170190
min,1.071186e+07,4.000000,7.0,31.000000,1.000000,3.000000,2.000000,1.000000,3.490000e+02,38.000000,...,1.000000,1.0,4.400000e+01,1.000000e+03,2016.0,1.610000e+02,19.920000,3.000000,6.037101e+13,-4.655420
25%,1.153830e+07,6.000000,7.0,261.000000,2.000000,4.000000,7.000000,1.000000,7.520500e+05,273.000000,...,1.000000,1.0,8.426500e+04,2.070000e+05,2016.0,8.550400e+04,2715.585000,14.000000,6.037311e+13,-0.024377
50%,1.253155e+07,6.000000,7.0,261.000000,2.000000,4.000000,7.000000,1.000000,1.497870e+06,515.000000,...,1.000000,1.0,1.364995e+05,3.589760e+05,2016.0,2.033835e+05,4450.720000,15.000000,6.037603e+13,0.006627
75%,1.421184e+07,6.000000,7.0,266.000000,7.000000,4.000000,7.000000,1.000000,2.240480e+06,796.500000,...,2.000000,1.0,2.187875e+05,5.690030e+05,2016.0,3.668022e+05,6927.800000,15.000000,6.059042e+13,0.039204
max,1.676893e+08,13.000000,7.0,275.000000,24.000000,4.000000,21.000000,13.000000,2.982274e+06,3560.000000,...,6.000000,1.0,1.142179e+07,4.906124e+07,2016.0,4.895220e+07,586639.300000,99.000000,4.830301e+14,5.262999


### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [10]:
def missing_values(df):
    df_nulls = pd.DataFrame({'num_rows_missing':df.isnull().sum(),'pct_rows_missing':df.isnull().sum()/len(df)*100})
    return df_nulls

In [11]:
df_nulls = missing_values(df)

In [12]:
df_nulls

,num_rows_missing,pct_rows_misisng
parcelid,0,0.000000
typeconstructiontypeid,77158,99.713104
storytypeid,77330,99.935384
propertylandusetypeid,0,0.000000
heatingorsystemtypeid,27941,36.108814
...,...,...
buildingclassdesc,77365,99.980615
heatingorsystemdesc,27941,36.108814
propertylandusedesc,0,0.000000
storydesc,77330,99.935384


In [14]:
df_nulls.pct_rows_misisng.describe()

count    68.000000
mean     48.566281
std      44.835208
min       0.000000
25%       0.002262
50%      48.094469
75%      98.162316
max      99.980615
Name: pct_rows_misisng, dtype: float64

Takeaways: drop high percentage columns and impute low percentage columns

### 4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

## Prepare

1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

The input:
- A dataframe
- A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
- A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
The output:
- The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
hint:
Look up the dropna documentation.
You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
Make use of inplace, i.e. inplace=True/False.
Decide how to handle the remaining missing values:

Fill with constant value.
Impute with mean, median, mode.
Drop row/column
wrangle_zillow.py

Functions of the work above needed to acquire and prepare a new sample of data.



In [18]:
df[df.propertylandusetypeid != 261].propertylandusedesc.value_counts()

Condominium                                   19294
Duplex (2 Units, Any Combination)              2009
Planned Unit Development                       1944
Quadruplex (4 Units, Any Combination)           727
Triplex (3 Units, Any Combination)              535
Cluster Home                                    333
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       58
Residential General                              37
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: propertylandusedesc, dtype: int64